In [18]:

# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [102]:

# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [103]:

# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])

In [104]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    #'vect__max_features': (500,1000),
    'clf__n_estimators':(50, 100,150),
    'clf__max_depth':(15,20,None)
}

# Fit
rand_search = RandomizedSearchCV(pipe,params, cv=5, n_jobs=-1, n_iter=12, verbose=1)
rand_search.fit(train['description'], train.ratingCategory)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.4min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [105]:
rand_search.best_score_

0.7210693903690824

In [8]:

# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [81]:
import re
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')



In [79]:

for row in train['description']:
    row = [x[1:] for x in row]
    

SyntaxError: 'return' outside function (<ipython-input-79-3cb1adf75e7b>, line 3)

In [82]:
train.description = [x[1:] for x in train.description]
    


In [106]:
train.head()

,id,description,ratingCategory
0,1321,"Sometimes, when whisky is batched, a few lefto...",1
1,3861,An uncommon exclusive bottling of a 6 year old...,0
2,655,This release is a port version of Amrut’s Inte...,1
3,555,This 41 year old single cask was aged in a she...,1
4,1965,"Quite herbal on the nose, with aromas of dried...",1


In [58]:
train['description'][0][1]

'S'

In [63]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [38]:

# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, train['ratingCategory'])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:   14.5s finished


AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [14]:
grid_search.best_score_

0.7308550274874085

In [ ]:

# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:

submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:

# Make Sure the Category is an Integer
submission.head()

In [ ]:
subNumber = 0

In [ ]:
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

In [84]:

import spacy
nlp = spacy.load("en_core_web_lg")

In [85]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [86]:
X = get_word_vectors(train['description'])

len(X) == len(train['description'])

True

In [87]:
X_test = get_word_vectors(test['description'])

In [88]:
rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [89]:
rfc.score(X, train['ratingCategory'])

1.0

In [90]:
rfc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0])

In [91]:
test['ratingCategory'] = rfc.predict(X_test)

In [92]:

test[['id', 'ratingCategory']].to_csv('testSolutionSubmission_spacy2.csv', header=True, index=False)